# Reproducing Haklidir & Haklidir

We are going to try to reproduce a paper on machine learning in a geothermal setting:

> Tut Haklidir, F.S., & Haklidir, M. (2019). Prediction of Reservoir Temperatures Using Hydrogeochemical Data, Western Anatolia Geothermal Systems (Turkey): A Machine Learning Approach. _Natural Resources Research_, **29**, 2333-2346. [DOI:10.1007/s11053-019-09596-0](https://link.springer.com/article/10.1007%2Fs11053-019-09596-0)

I first heard of this work at the 2020 World Geothermal Congress (which was in 2021) during this talk: https://www.youtube.com/watch?v=-Y0fb23FDzI. The talk frames the task as a classification task, predicting classes of temperature (high, medium, low). But it seems more sensible to cast this as a regression, and that's what the paper does. As far as I can tell, the two papers use the same dataset (but it's hard to be certain).

Plan:

- Load the data.
- Try to reproduce the figures in the journal article.
- Try to reproduce the results in the conference paper.

## Load the data

You can see the process of loading and cleaning the original data from the paper [in this original 'reproduction'](https://github.com/softwareunderground/repro-zoo/blob/main/haklidir-and-haklidir-2019/Reproducing_models.ipynb) at the Software Underground Repro Zoo repo. I'll spare you the grisly details here :)

Load the cleaned data:

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/scienxlab/datasets/refs/heads/main/misc/haklidir-and-haklidir-2019.csv'
df = pd.read_csv(url)
df.head()

**Note that there are two samples labeled 35 and no sample 63. This is how the dataset was published.**

## Question about the temperatures

I believe the adiabatic (i.e. with steam loss, for open systems) silica geothemometry equation for high-temperature systems (over 150 deg C) is:

$$ T = \frac{1522}{5.75 - log(S)} - 273.15  $$

where $S$ is in mg/kg. There are adjustments to the parameters in this equation for low temperature systems.

The problem here is that if T was calculated from SiO<sub>2</sub> then all we can achieve with machine learning is the discovery of that linear relationship. So I'm inclined to throw out the data from thermal springs — unfortunately it's a lot of data.

In [ ]:
df.loc[df['Source type']=='Thermal spring'].count()['Sample label']

So only 23 samples out of 83 are from wells.

Let's compute this temperature and see if we can figure out where the numbers in the temperature column came from.

In [ ]:
import numpy as np

def geotemp(S: float) -> float:
    """
    Fournier 1977 equation for temperature from silica,
    without steam loss.
    """
    return 1522 / (5.75 - np.log10(S)) - 273.15

In [ ]:
# Add geothermometry as feature.
df['Geothermometry'] = df['SiO2 (mg/l)'].map(geotemp)

In [ ]:
import matplotlib.pyplot as plt

is_spring = lambda x: x=='Thermal spring'

plt.scatter(df['Geothermometry'],
            df['Reservoir temperature (°C)'],
            c=is_spring(df['Source type']), cmap='bwr',
            s=30*((1+(df['Dataset']!="Training"))/2),  # Validation points are larger.
            alpha=((1+(df['Dataset']=="Training"))/2)  # Validation points are semi-transp.
           )

Wow.

So to obtain the reservoir temperature, which is the target of their machine learning task, the authors applied this equation to the silica concentration even though:

1. They are trying to predict temperature from the geochemistry.
1. The equation is not appropriate for temperatures below about 150 degC. (This doesn't really matter though, becase of the first point.)

This undermines the entire piece of work, limiting it to the 23 samples from wells that have independent measures of temperature.

## Create `X` and `y` and split the data

The authors state in Figure 4 they were using **Temperature** as input, as well as output. But they can't have been doing that because it would lead to perfect predictions. So I'm not sure what that feature is and assume it was a mistake to indicate it as an input.

They also state that they did use SiO<sub>2</sub>, even though it was used to compute the target. I've checked the models with and without this feature, and whether I use it or not, I cannot reproduce their results, so it's hard to say. I hope they did not use it, but I suspect they did (indeed, the errors I get are more in line with theirs if I do use it).

In [ ]:
features = ['pH', 'EC (microS/cm)', 'K (mg/l)', 'Na (mg/l)', 'Boron (mg/l)', 'SiO2 (mg/l)', 'Cl (mg/l)']
target = 'Reservoir temperature (°C)'

# Conditions.
train = df['Dataset']=='Training'
val = df['Dataset']=='Validation'
well = df['Source type']=='Well'

X_train = df.loc[train & well, features]
y_train = df.loc[train & well, target]
X_val = df.loc[val & well, features]
y_val = df.loc[val & well, target]

In [ ]:
X_train.shape, X_val.shape

## Standardize the data

The authors do not say if they did this or not. Since the scales of the inputs vary so widely, and since we're using models like SVM and a neural network, it's an important step. For now we'll just scale the inputs, but the neural net will probably perform better with scaled output as well.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

### Train and predict: Linear regression

The authors don't say if they were using regularization or not, but we want the best result possible so we'll use it. I tuned it manually.

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge

model = Ridge(alpha=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
y_hat = model.predict(X_train)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)

print(f'RMSE for linear regression: {rmse:.1f} deg C')
print(f'MAE for linear regression: {mae:.1f} deg C')

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from urllib.request import urlopen

fig7a = 'https://raw.githubusercontent.com/softwareunderground/repro-zoo/main/haklidir-and-haklidir-2019/Fig7a.png'

im = np.array(Image.open(urlopen(fig7a)))
extent = [-3.8, 296, 9, 295]

plt.figure(figsize=(10, 10))
plt.imshow(im, origin='upper', extent=extent)
plt.scatter(y_train, y_hat, alpha=0.5, s=70, c='c')
plt.scatter(y_val, y_pred, c='C1', alpha=0.75)
plt.axis('equal')
plt.axis('off')
plt.show()

### Train and predict: linear SVM

Again the authors don't say if they applied regularization or not. We'll use it (it's on by default).

In [ ]:
from sklearn.svm import SVR

svm = SVR(kernel='linear', C=1)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_val)
y_hat = svm.predict(X_train)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)

print(f'RMSE for linear SVM: {rmse:.1f} deg C')
print(f'MAE for linear SVM: {mae:.1f} deg C')

In [ ]:
fig7b = "https://raw.githubusercontent.com/softwareunderground/repro-zoo/main/haklidir-and-haklidir-2019/Fig7b.png"
im = np.array(Image.open(urlopen(fig7b)))
extent = [5, 270, 14, 275]

plt.figure(figsize=(10, 10))
plt.imshow(im, origin='upper', extent=extent)
plt.scatter(y_train, y_hat, alpha=0.4, s=80, c='c')
plt.scatter(y_val, y_pred, c='C1', alpha=0.75)
plt.axis('equal')
plt.axis('off')
plt.show()

### Non-linear SVM

I was curious how this would do in comparison.

Let's tune the regularization parameter `C` a bit to get a reasonable fit without overtraining:

In [ ]:
Cs = np.logspace(-1, 4, 11)
trains, vals = [], []

for C in Cs:
    svm = SVR(C=C)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_val)
    y_hat = svm.predict(X_train)

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_train = np.sqrt(mean_squared_error(y_hat, y_train))

    trains.append(rmse_train)
    vals.append(rmse)

plt.figure(figsize=(10, 6))
plt.plot(Cs, trains, 'o-')
plt.plot(Cs, vals, 'o-')
plt.xscale('log')
plt.grid(c='k', alpha=0.2)

In [ ]:
Cbest = Cs[np.argmin(vals)]
Cbest

In [ ]:
svm = SVR(C=Cbest)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_val)
y_hat = svm.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)

print(f'RMSE for non-linear SVM: {rmse:.1f} deg C')
print(f'MAE for non-linear SVM: {mae:.1f} deg C')

In [ ]:
extent = [5, 270, 14, 275]

plt.figure(figsize=(10, 10))
plt.imshow(im, origin='upper', extent=extent)
plt.scatter(y_train, y_hat, alpha=0.4, s=80, c='c')
plt.scatter(y_val, y_pred, c='C1')
plt.axis('equal')
plt.axis('off')
plt.show()

Clearly much better (and the fit to the training data is, not surprisingly, almost perfect).

### Train and predict: Neural network

First let's scale the output:

In [ ]:
yscaler = StandardScaler()
yscaler.fit(y_train.values.reshape(-1, 1))
y_train_ = yscaler.transform(y_train.values.reshape(-1, 1)).squeeze()
y_val_ = yscaler.transform(y_val.values.reshape(-1, 1)).squeeze()

The authors do not say (in the talk, or in the paper) what the architecture of their network is, although the figure implies that it has 2 hidden layers (i.e. not including input or output).

They also don't mention the activation function, optimization strategy, regularization applied... or basically give any details at all.

In [ ]:
from sklearn.neural_network import MLPRegressor

net = MLPRegressor(hidden_layer_sizes=(5, 5),
                   max_iter=500,
                   random_state=42,
                  )
net.fit(X_train, y_train_)
y_pred_ = net.predict(X_val)
y_hat_ = net.predict(X_train)

# It would be much smarter to use TransformedTargetRegressor here!
y_pred = yscaler.inverse_transform(y_pred_.reshape(-1, 1))
y_hat = yscaler.inverse_transform(y_hat_.reshape(-1, 1))

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)

print(f'RMSE for neural net: {rmse:.1f} deg C')
print(f'MAE for neural net: {mae:.1f} deg C')

In [ ]:
fig7c = "https://raw.githubusercontent.com/softwareunderground/repro-zoo/main/haklidir-and-haklidir-2019/Fig7c.png"
im = np.array(Image.open(urlopen(fig7c)))
extent = [22, 264, 37, 251]

plt.figure(figsize=(10, 10))
plt.imshow(im, origin='upper', extent=extent)
plt.scatter(y_train, y_hat, alpha=0.4, s=80, c='c')
plt.scatter(y_val, y_pred, c='C1', alpha=0.75)
plt.plot([60, 240], [60, 240], c='r')
plt.axis('equal')
plt.axis('off')
plt.show()

Note that the $x = y$ line in the published plot is incorrect so I plotted a new one (red).

Safe to say that our results do not agree.

---

&copy; 2023 Matt Hall, licensed CC BY